<a href="https://colab.research.google.com/github/vlastimiltetour/cabin_log_scraper/blob/main/Log_cabin_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files

# Upload the .mhtml file
uploaded = files.upload()

Saving main.mhtml to main.mhtml


In [5]:
import email
import base64
from bs4 import BeautifulSoup

def parse_cabin_file(file_path):

  # Replace with your uploaded file's name
  file_path = '/content/main.mhtml'

  # Read the .mhtml file
  with open(file_path, 'rb') as file:
      mhtml_data = file.read()

  # Parse the .mhtml file as an email message
  msg = email.message_from_bytes(mhtml_data)

  # Iterate through the MIME parts
  for part in msg.walk():
      content_type = part.get_content_type()
      if content_type == 'text/html':  # Find the HTML part
          # Decode the HTML content
          html_content = part.get_payload(decode=True).decode('utf-8')
          break

  # Parse the HTML with BeautifulSoup
  soup = BeautifulSoup(html_content, 'html.parser')
  return soup

  # Example: Extract all text from the HTML


In [18]:
from types import NoneType
def return_cabin_overview():
  url_list = ['/content/main.mhtml']
  count = 0


  for url in url_list:

    soup = parse_cabin_file(url)

    cabin_root = soup.find_all("li",class_="MuiGrid-root MuiGrid-item css-j4rfqf")
    #id="region-tip-item-1-1707319884">

    for cabin in cabin_root:
      try:
        title_tag = cabin.find("p", class_="MuiTypography-root MuiTypography-body1 css-mjbyyl")
        title = title_tag.text if title_tag else 'title not found'

        location = cabin.find("p", class_="MuiTypography-root MuiTypography-body1 css-mjbyyl").find_next_sibling("p").text
        price = cabin.find("p", class_="MuiTypography-root MuiTypography-body1 css-2bjqts").text

        print(title, location, price)
        count += 1
      except Exception as e:
        print('this error has been found', e)
    return len(cabin_root), count


print(return_cabin_overview())

Prodej rodinného domu 270 m², pozemek 1331 m² Vlčkovice v Podkrkonoší - Horní Vlčkovice 1 690 000 Kč
Prodej chalupy 360 m², pozemek 198 m² Pec pod Sněžkou - Velká Úpa 20 000 000 Kč
Prodej chalupy 270 m², pozemek 1746 m² Dolní Dvůr Cena na vyžádání
Prodej chalupy 140 m², pozemek 524 m² Spojovací, Svoboda nad Úpou 5 200 000 Kč
Prodej chalupy 200 m², pozemek 1591 m² Dvůr Králové nad Labem - Lipnice 2 490 000 Kč
Prodej chalupy 350 m², pozemek 17218 m² Dolní Dvůr 11 990 000 Kč
Prodej chalupy 50 m², pozemek 2400 m² Mostek - Debrné 5 100 000 Kč
Prodej chalupy 181 m², pozemek 701 m² Žacléř - Prkenný Důl 10 990 000 Kč
Prodej chalupy 533 m², pozemek 7025 m² Pec pod Sněžkou - Velká Úpa 24 990 000 Kč
Prodej chalupy 80 m², pozemek 770 m² Chotěvice 2 200 000 Kč
Prodej chalupy 160 m², pozemek 1439 m² Dolní Branná 2 499 000 Kč
Prodej chalupy 143 m², pozemek 1500 m² Zlatá Olešnice 2 990 000 Kč
this error has been found 'NoneType' object has no attribute 'find_next_sibling'
Prodej chalupy 375 m², pozeme

In [4]:
def return_cabin_details():
  url_list = ['/content/web.mhtml']

  for url in url_list:
    soup = parse_cabin_file(url)

    title = soup.find("h1", class_="MuiTypography-root MuiTypography-body1 css-kttn7v").text
    price = soup.find("p", class_="MuiTypography-root MuiTypography-body1 css-117xoa7").text

    print(title, price)
    return


print(return_cabin_details())

AttributeError: 'NoneType' object has no attribute 'text'

In [ ]:
import requests
from bs4 import BeautifulSoup
#url = 'https://www.sreality.cz/detail/prodej/dum/chalupa/trstenice-trstenice-/3942568524'
url = '/content/web.mhtml'

page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")
print(soup)